# MNIST 손글씨 - CNN v4
- 2 Conv2D layers ==> 2Conv2D with padding layers
- 1 MaxPooling2D layer
- 1 Dropout(0.25) layer
- 1 Fully Connected Network layer
- 1 Dropout(0.5) layer
- 1 Output layer

In [ ]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

### 데이터 전처리

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# X data : 0 ~ 1 사이의 값으로 정규화 & 3차원 → 4차원
X_train = X_train.reshape(-1, 28, 28, 1) / 255.
X_test = X_test.reshape(-1, 28, 28, 1) / 255.

In [ ]:
# One-hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

### 모델 정의/설정/학습/평가

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3,3), padding='same', input_shape=(28,28,1), activation='relu'),    # 마스크(필터)의 종류 32가지 사용
    Conv2D(64, (3,3), padding='same', activation='relu'), # padding: input의 크기 만큼 이미지의 상하좌우에 '0'으로 된 pad가 둘러짐
    MaxPooling2D(),                         # MaxPooling2D(pool_size=(2,2))
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 512)               6423040   
                                                        

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_path = 'best-mnist-cnn-v4.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
early_stop = EarlyStopping(patience=10)

In [ ]:
hist = model.fit(
    X_train, Y_train, validation_split=0.2,
    epochs=100, batch_size=200,
    callbacks=[checkpoint, early_stop], verbose=0)


Epoch 00001: val_loss improved from inf to 0.05631, saving model to best-mnist-cnn-v4.h5

Epoch 00002: val_loss improved from 0.05631 to 0.04948, saving model to best-mnist-cnn-v4.h5

Epoch 00003: val_loss improved from 0.04948 to 0.04242, saving model to best-mnist-cnn-v4.h5

Epoch 00004: val_loss improved from 0.04242 to 0.03519, saving model to best-mnist-cnn-v4.h5

Epoch 00005: val_loss did not improve from 0.03519

Epoch 00006: val_loss did not improve from 0.03519

Epoch 00007: val_loss did not improve from 0.03519

Epoch 00008: val_loss did not improve from 0.03519

Epoch 00009: val_loss improved from 0.03519 to 0.03338, saving model to best-mnist-cnn-v4.h5

Epoch 00010: val_loss did not improve from 0.03338

Epoch 00011: val_loss did not improve from 0.03338

Epoch 00012: val_loss did not improve from 0.03338

Epoch 00013: val_loss did not improve from 0.03338

Epoch 00014: val_loss did not improve from 0.03338

Epoch 00015: val_loss did not improve from 0.03338

Epoch 00016: 

In [ ]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0261 - accuracy: 0.9918


[0.026117144152522087, 0.9918000102043152]